In [1]:
import glob
import json
import timeit
import os
import numpy as np


In [2]:
data_folder = "../../data/"
files_to_load = glob.glob(data_folder + "*_vec2raster_ne.json")
files_to_load


['../../data/precip_vec2raster_ne.json', '../../data/tavg_vec2raster_ne.json']

In [3]:
with open('../../data/tavg_vec2raster_ne.json', 'r') as load_from_json:
    tavg_json = json.load(load_from_json)

with open('../../data/precip_vec2raster_ne.json', 'r') as load_from_json:
    precip_json = json.load(load_from_json)


In [4]:
print(len(tavg_json["features"]))
print(len(precip_json["features"]))


805197
805197


In [5]:
tavg_json.keys()


dict_keys(['type', 'features'])

In [6]:
tavg_json['type']

'FeatureCollection'

In [7]:
tavg = tavg_json["features"][0]
tavg


{'id': '0',
 'type': 'Feature',
 'properties': {'id': '110',
  'tavg_value': -25.803125381469727,
  'tavg_color': '#6bacd0'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-180.0, 71.66666666666669],
    [-179.83333333333334, 71.66666666666669],
    [-179.83333333333334, 71.50000000000001],
    [-180.0, 71.50000000000001],
    [-180.0, 71.66666666666669]]]}}

In [8]:
start = timeit.default_timer()
#---------------------------------------------------------------------------

combined_list = []

for index in range(0, len(tavg_json["features"])):
    if(index % 100000 == 0 and index != 0):
        print(f"Done merging {index} features.")
    
    tavg = tavg_json["features"][index]
    precip = precip_json["features"][index]

    assert(tavg['id'] == precip['id'])

    combined = {
        'id': index,
        'type': tavg['type'],
        'properties': {
            'id': tavg['properties']['id'],
            'tavg_value': tavg['properties']['tavg_value'],
            'tavg_color': tavg['properties']['tavg_color'],
            'precip_value': precip['properties']['precip_value'],
            'precip_color': precip['properties']['precip_color'],
        },
        'geometry': tavg['geometry']
    }
    combined_list.append(combined)

#---------------------------------------------------------------------------
end = timeit.default_timer()
print()
print(f"Time to complete: {(end - start):.5f}s")


Done merging 100000 features.
Done merging 200000 features.
Done merging 300000 features.
Done merging 400000 features.
Done merging 500000 features.
Done merging 600000 features.
Done merging 700000 features.
Done merging 800000 features.

Time to complete: 0.58840s


In [9]:
print(len(combined_list))
assert(len(combined_list) == len(tavg_json["features"]))


805197


In [10]:
"geometry" in combined_list[0].keys()


True

In [11]:
combined_list[0]["properties"]


{'id': '110',
 'tavg_value': -25.803125381469727,
 'tavg_color': '#6bacd0',
 'precip_value': 6.0,
 'precip_color': '#f5f9fe'}

In [12]:
# create a dict that we can save to a json file that still maintains
# header information of input file
combined_dict = dict()
combined_dict["type"] = tavg_json["type"]
combined_dict["variables"] = ["tavg", "vpd"]
combined_dict["features"] = combined_list


In [13]:
# https://stackoverflow.com/questions/49865442/write-into-a-js-file-without-passing-in-string-quotes-in-python
# varname = "const raster_grid = "
# with open('../../data/raster_grid_ne.js', 'w') as outfile:
#     outfile.write(varname)
#     # write the JSON value after the variable declaration
#     outfile.write(json.dumps(combined_dict))

# with open('../../data/raster_grid_ne.json', 'w') as outfile:
#     outfile.write(json.dumps(combined_dict))


In [14]:
# the individual files are around ~200-300 MB
# the merged file for 2 variables is ~342 MB
# however, it does mean that our file will always be limited by the geometry field
filesize = np.round(os.path.getsize("../../data/raster_grid_ne.js") / 1e6, 3)
print(f"The file is {filesize} MB")


The file is 342.092 MB
